In [1]:
%pip install dnspython

import dns.resolver
import time
import subprocess
import platform


def measure_dns_resolution_time(domain, record_type='A'):
    """Measures the time taken to resolve a DNS record."""

    resolver = dns.resolver.Resolver()
    start_time = time.perf_counter()  # Use perf_counter for accurate timing
    try:
        answers = resolver.resolve(domain, record_type)
        end_time = time.perf_counter()
        resolution_time = end_time - start_time
        print(f"Resolved {domain} ({record_type}) in {resolution_time:.4f} seconds")
        for rdata in answers:
            print(f"  {rdata}")
        return resolution_time

    except dns.exception.DNSException as e:
        print(f"Error resolving {domain}: {e}")
        return None



def traceroute_to_server(hostname):
    """Performs a traceroute to the given hostname using the system's traceroute."""
    try:
        if platform.system().lower() == 'windows':
            process = subprocess.run(['tracert', hostname], capture_output=True, text=True, check=True)
        else:
            process = subprocess.run(['traceroute', hostname], capture_output=True, text=True, check=True)
        output = process.stdout
        print(output)  
        # You can further process the output here (e.g., extract hop timings)

    except subprocess.CalledProcessError as e:
        print(f"Traceroute to {hostname} failed: {e.stderr if e.stderr else e}")



def get_authoritative_nameservers(domain):

    resolver = dns.resolver.Resolver()

    try:
      answers = resolver.resolve(domain, 'NS')
      nameservers = [ns.to_text() for ns in answers]
      print(f"Authoritative nameservers for {domain}:")
      for ns in nameservers:
          print(ns)
      return nameservers

    except dns.exception.DNSException as e:
        print(f"Error getting nameservers for {domain}: {e}")
        return []

# Example Usage
domain = 'google.com'  # Replace with the domain you want to test
measure_dns_resolution_time(domain)
measure_dns_resolution_time(domain, 'MX')
get_authoritative_nameservers(domain)
traceroute_to_server(domain)

Note: you may need to restart the kernel to use updated packages.
Resolved google.com (A) in 2.0406 seconds
  142.250.205.238
Resolved google.com (MX) in 2.0373 seconds
  10 smtp.google.com.
Authoritative nameservers for google.com:
ns1.google.com.
ns4.google.com.
ns3.google.com.
ns2.google.com.

Tracing route to google.com [142.250.205.238]
over a maximum of 30 hops:

  1     2 ms     2 ms     1 ms  172.16.0.1 
  2     4 ms     4 ms     4 ms  10.1.21.254 
  3    22 ms     3 ms     4 ms  192.168.40.100 
  4    30 ms    37 ms    20 ms  103.115.236.17 
  5    33 ms    17 ms    17 ms  103.173.252.185 
  6    36 ms    15 ms    37 ms  72.14.202.228 
  7    30 ms    77 ms    24 ms  142.251.230.177 
  8    40 ms    50 ms    34 ms  142.251.60.187 
  9    68 ms    33 ms    48 ms  maa05s28-in-f14.1e100.net [142.250.205.238] 

Trace complete.

